<a href="https://colab.research.google.com/github/Jonghun-Han/H.Engineering-report-Jonghun-Han/blob/main/offline_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

#link to mock data set from GitHub
url = "https://raw.githubusercontent.com/Jonghun-Han/H.Engineering-report-Jonghun-Han/refs/heads/main/mock_mobility_data.csv"

#loading the CSV file from GitHub
df = pd.read_csv(url)

#to show a preview of data
df.head()

,id,name,disability_type,location,needs,synced
0,1,Ram,amputation,"""Baneshwor, Kathmandu""",prosthetic leg,0
1,2,Laxmi,arthritis,"""Thimi, Bhaktapur""","""walker, pain relief medication""",1
2,3,Krishna,spinal cord injury,"""Kirtipur, Kathmandu""","""wheelchair, urinary care supplies, accessible...",1
3,4,Dhan,cerebral palsy,"""Baneshwor, Kathmandu""","""wheelchair, emergency therapy, accessible she...",0
4,5,Rima,sprained ankle,"""Gongabu, Kathmandu""","""crutches, pain relief medication""",0


In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/Jonghun-Han/H.Engineering-report-Jonghun-Han/refs/heads/main/mock_mobility_data.csv"

df = pd.read_csv(url)

import sqlite3
import requests

#creating local SQLite database
conn = sqlite3.connect("aidlink_Nepal.db")
df.to_sql("aidlink_Nepal", conn, if_exists="replace", index=False)


#Extract unsynced data, showing the fieldworkers for which data will be sent to the server
cursor = conn.cursor()
print("Unsynced data before sync:")
for row in cursor.execute("select * from aidlink_Nepal where synced = 0"):
  print(row)

cursor.execute("select * from aidlink_Nepal where synced = 0")
unsynced_rows = cursor.fetchall()

payload = [
    {"id": row[0], "name": row[1], "disability_type": row[2], "location": row[3], "needs": row[4]}
    for row in unsynced_rows
]

#sending the data to the test server httpbin.org, if status code 200 appears, synce is successful
try:
    response = requests.post("https://httpbin.org/post", json=payload)
    if response.status_code == 200:
        print(" Sync successful!! Updating local DB...")
        cursor.execute("update aidlink_Nepal SET synced = 1 where synced = 0")
        conn.commit()
    else:
        print(f" Sync failed with status: {response.status_code}")
except Exception as e:
    print(" Sync failed due to error:", e)

#checking if synced is all changed to 1
cursor.execute("select * from aidlink_Nepal")
print("\n All data after sync:")
for row in cursor.fetchall():
    print(row)

conn.close()


Unsynced data before sync:
(1, 'Ram', 'amputation', '"Baneshwor, Kathmandu"', 'prosthetic leg', 0)
(4, 'Dhan', 'cerebral palsy', '"Baneshwor, Kathmandu"', '"wheelchair, emergency therapy, accessible shelter"', 0)
(5, 'Rima', 'sprained ankle', '"Gongabu, Kathmandu"', '"crutches, pain relief medication"', 0)
(6, 'Prakash', 'blindness', '"Baneshwor, Kathmandu"', '"cane, accessible shelter"', 0)
 Sync successful!! Updating local DB...

 All data after sync:
(1, 'Ram', 'amputation', '"Baneshwor, Kathmandu"', 'prosthetic leg', 1)
(2, 'Laxmi', 'arthritis', '"Thimi, Bhaktapur"', '"walker, pain relief medication"', 1)
(3, 'Krishna', 'spinal cord injury', '"Kirtipur, Kathmandu"', '"wheelchair, urinary care supplies, accessible shelter"', 1)
(4, 'Dhan', 'cerebral palsy', '"Baneshwor, Kathmandu"', '"wheelchair, emergency therapy, accessible shelter"', 1)
(5, 'Rima', 'sprained ankle', '"Gongabu, Kathmandu"', '"crutches, pain relief medication"', 1)
(6, 'Prakash', 'blindness', '"Baneshwor, Kathmandu